In [122]:
import tensorflow as tf
import larq as lq
import numpy as np
import pandas as pd
import seaborn as sns
from binaryflow import quantizers
from binaryflow.layers import ABCNet,XnorNet,BinaryNet
from binaryflow.block import BiRealNet
from binaryflow.layers.normalization import *
from contextlib import redirect_stdout
import json


In [116]:
  data_format="channels_last"
    
(X_train,y_train),(X_test,y_test)=tf.keras.datasets.mnist.load_data()
X_train=X_train.astype(dtype=np.float32)
y_train=tf.one_hot(y_train,10)
X_test=X_test.astype(dtype=np.float32)
X_train=X_train.reshape([*X_train.shape,1])
X_test=X_test.reshape([*X_test.shape,1])
# All quantized layers except the first will use the same options

abc_args = dict(
              kernel_quantizers=quantizers.ShiftedSteSign,
              input_quantizers=quantizers.ShiftedSteSign,
              kernel_constraint="weight_clip",
              #kernel_params={"mu_initializer":tf.keras.initializers.RandomNormal(0,0.05)},
              use_bias=False
              )

bnn_args=dict(kernel_quantizer="ste_sign",
              input_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              use_bias=False)

Args=[bnn_args]*2+[abc_args]

Archs=[BinaryNet.QuantDense,XnorNet.ScaledQuantDense,ABCNet.ABCDense]
Names=["BinaryNet","XnorNet","ABCNet"]



Dense=BiRealNet.BiRealDense
Conv2D=XnorNet.ScaledQuantPlusConv2D

models=[]

model=tf.keras.models.Sequential([
tf.keras.layers.GaussianNoise(stddev=4),
ImageNormalizationLayer(),
#tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),

#Conv2D(100,kernel=(3,3),mode=2,**bnn_args,activation="relu"),
tf.keras.layers.Flatten(),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),

ABCNet.ABCDense(1024,activation="relu",**abc_args),
#tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(estimator_type=ABCNet.ABCDense,**abc_args).add(tf.keras.layers.BatchNormalization(momentum=0.999,scale=False)),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
tf.keras.layers.Dense(10),
tf.keras.layers.Activation("softmax")
])
models.append(model)
model.compile(
tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
loss="categorical_crossentropy",
metrics=["accuracy"],
)


trained_model = model.fit(
    X_train, 
    y_train,
    batch_size=128, 
    epochs=1,
    validation_data=(X_test, tf.one_hot(y_test, 10)),
    shuffle=True
)

            



  1/469 [..............................] - ETA: 16:47 - loss: 2.5439 - accuracy: 0.0938

KeyboardInterrupt: 

In [42]:
model2=tf.keras.models.load_model("local/Test")

In [133]:
class CallbackRegistrable():
    def __init__(self):
        self.onFitCallbacks=[]
        self.beforeFitCallbacks=[]
        self.onFitResults={}
        self.beforeFitResults={}
    
    def registerOnFitCallback(self,name:str,callbackFn):
        self.onFitCallbacks.append((name,callbackFn))
        self.onFitResults[name]=[]

    def registerBeforeFitCallback(self,name:str,callbackFn):
        self.beforeFitCallbacks.append((name,callbackFn))
        self.beforeFitResults[name]=[] 
    
    def onModelFit(self,index,model,name,count):
        for callbackName,callback in self.onFitCallbacks:
            self.onFitResults[callbackName].append(callback(index,model,name,count))
            
    def beforeModelFit(self,index,model,name,count):
        for callbackName,callback in self.beforeFitCallbacks:
            self.beforeFitResults[callbackName].append(callback(index,model,name,count))
    

class TrainingSimulation(CallbackRegistrable):
    def __init__(self,name:str,Dense,count:int,architecture="standard",**dense_kwargs):
        super(TrainingSimulation,self).__init__()
        self.name=name
        self.count=count
        if architecture == "standard":
            architectureCaller=MNISTSimulation.standardArchitecture
        elif achitecture == "refined":
            architectureCaller=MNISTSimulation.refinedArchitecture
        else:
            raise RuntimeError(f"Unrecognized architecture {architecture}")
        self.models=[architectureCaller(Dense,**dense_kwargs) for i in range(count)]
    
    @staticmethod
    def standardArchitecture(Dense,**dense_kwargs):
            return tf.keras.Sequential([tf.keras.layers.GaussianNoise(stddev=4),
ImageNormalizationLayer(),
#tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),

#Conv2D(100,kernel=(3,3),mode=2,**bnn_args,activation="relu"),
tf.keras.layers.Flatten(),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(1024,activation="relu",**dense_kwargs),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),            
Dense(1024,activation="relu",**dense_kwargs),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(10,**dense_kwargs),
tf.keras.layers.Activation("softmax")
])
        
    @staticmethod
    def refinedArchitecture(Dense,**dense_kwargs):
        return tf.keras.Sequential([tf.keras.layers.GaussianNoise(stddev=4),
ImageNormalizationLayer(),
#tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),

#Conv2D(100,kernel=(3,3),mode=2,**bnn_args,activation="relu"),
tf.keras.layers.Flatten(),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(1024,activation="relu",**dense_kwargs),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),            
Dense(1024,activation="relu",**dense_kwargs),
tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
Dense(10),
tf.keras.layers.Activation("softmax")
])
    
    def compile(self,optimizer="adam",loss="categorical_crossentropy",metrics=None):
        if metrics is None:
            metrics=["accuracy"]
        for model in self.models:
            model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

    
    def fit(self,X_train,y_train,X_test,y_test,batch_size,epochs,shuffle=True):
        for index,model in enumerate(self.models):
            self.beforeModelFit(index,model,self.name,self.count)
            model.fit(X_train,y_train,batch_size=batch_size,validation_data=(X_test,y_test),shuffle=shuffle,epochs=epochs)
            self.onModelFit(index,model,self.name,self.count)
    
    def generateSummary(self):
        U=pd.DataFrame()
        for index,model in enumerate(self.models):
            V=pd.DataFrame.from_dict(model.history.history)
            V["index"]=index
            V["epoch"]=np.arange(V.shape[0])+1
            U=pd.concat([U,V],ignore_index=True)
        U["name"]=self.name
        return U
    
class TrainingSimulations():
    def __init__(self,name,simulations=None):
        self.name=name
        if simulations is None:
            self.simulations=[]
        else:
            self.simulations=simulations
        self.beforeSimulationCallbacks={}
        self.onSimulationCallbacks={}
        
    def add(self,simulation:MNISTSimulation):
        self.simulations.append(simulation)

    def compile(self,optimizer="adam",loss="categorical_crossentropy",metrics=None):
        for simulation in self.simulations:
            simulation.compile(optimizer,loss,metrics)
    
    def fit(self,X_train,y_train,X_test,y_test,batch_size,epochs,shuffle=True):
        for index,simulation in enumerate(self.simulations):
            self.beforeSimulation(index,simulation,self.name)
            simulation.fit(X_train,y_train,X_test,y_test,batch_size,epochs,shuffle)
            self.onSimulation(index,simulation,self.name)
            
    def registerBeforeSimulationCallback(self,name,callbackFn):
        self.beforeSimulationCallbacks[name]=callbackFn
        
    def registerOnSimulationCallback(self,name,callbackFn):
        self.onSimulationCallbacks[name]=callbackFn
        
    def onSimulation(self,index,simulation,name):
        for callbackName in self.onSimulationCallbacks:
            self.onSimulationCallbacks[callbackName](index,simulation,name)
        
    def beforeSimulation(self,index,simulation,name):
        for callbackName in self.beforeSimulationCallbacks:
            self.beforeSimulationCallbacks[callbackName](index,simulation,name)

    def registerOnFitCallback(self,name:str,callbackFn):
        for simulation in self.simulations:
            simulation.registerOnFitCallback(name,callbackFn)

    def registerBeforeFitCallback(self,name:str,callbackFn):
        for simulation in self.simulations:
            simulation.registerBeforeFitCallback(name,callbackFn)
    
    def generateSummary(self):
        return pd.concat([ simulation.generateSummary() for simulation in self.simulations],ignore_index=True)
    

In [137]:
count=3
epochs=1


def outputSimulationIndex(index,model,name,count):
    print(f"Simulation {index+1}/{count} of {name}. on MNIST")

    
def outputNewSimulation(index,simulation,name):
    print(f"Beginning Simulation of {simulation.name}")



BinaryNetSimulation=TrainingSimulation("BinaryNet",BinaryNet.QuantDense,count,**bnn_args)
XnorNetSimulation=TrainingSimulation("XnorNet",XnorNet.XnorDense,count,**bnn_args)
ABCNetSimulation=TrainingSimulation("ABCNet",ABCNet.ABCDense,count,**abc_args)

MNISTSimulations=TrainingSimulations("MNIST",[BinaryNetSimulation,XnorNetSimulation,ABCNetSimulation])

MNISTSimulations.compile("adam","categorical_crossentropy",["accuracy"])
MNISTSimulations.registerBeforeFitCallback("LOG_SIMULATION",outputSimulationIndex)
MNISTSimulations.registerBeforeSimulationCallback("LOG_EXPERIENCE",outputNewSimulation)
MNISTSimulations.fit(X_train,y_train,X_test,tf.one_hot(y_test,10),batch_size=128,epochs=1,shuffle=True)
results=MNISTSimulations.generateSummary()
results.save("local/simulation_results.csv")

Beginning Simulation of BinaryNet
Simulation 1/3 of BinaryNet. on MNIST
469/469 [==============================] - 8s 16ms/step - loss: 6.4274 - accuracy: 0.8206 - val_loss: 10.8041 - val_accuracy: 0.7501
Simulation 2/3 of BinaryNet. on MNIST
469/469 [==============================] - 8s 16ms/step - loss: 6.6137 - accuracy: 0.8166 - val_loss: 10.3502 - val_accuracy: 0.7317
Simulation 3/3 of BinaryNet. on MNIST
469/469 [==============================] - 8s 16ms/step - loss: 6.5432 - accuracy: 0.8162 - val_loss: 9.8482 - val_accuracy: 0.7405
Beginning Simulation of XnorNet
Simulation 1/3 of XnorNet. on MNIST
469/469 [==============================] - 10s 19ms/step - loss: 0.3767 - accuracy: 0.8830 - val_loss: 1.4758 - val_accuracy: 0.7607
Simulation 2/3 of XnorNet. on MNIST
469/469 [==============================] - 10s 19ms/step - loss: 0.3709 - accuracy: 0.8848 - val_loss: 1.4527 - val_accuracy: 0.7200
Simulation 3/3 of XnorNet. on MNIST
469/469 [==============================] - 10s 1

AttributeError: 'DataFrame' object has no attribute 'save'

In [138]:
results

,loss,accuracy,val_loss,val_accuracy,index,epoch,name
0,6.427420,0.820600,10.804111,0.7501,0,1,BinaryNet
1,6.613717,0.816583,10.350192,0.7317,1,1,BinaryNet
2,6.543233,0.816233,9.848212,0.7405,2,1,BinaryNet
3,0.376703,0.883033,1.475776,0.7607,0,1,XnorNet
4,0.370865,0.884767,1.452742,0.7200,1,1,XnorNet
5,0.373086,0.883817,1.519054,0.7430,2,1,XnorNet
6,0.637319,0.795933,1.995327,0.3063,0,1,ABCNet
7,0.482756,0.850400,0.581381,0.8338,1,1,ABCNet
8,0.593693,0.811550,1.062044,0.7523,2,1,ABCNet


(0, 0)